In [1]:
import numpy as np
np.set_printoptions(linewidth=150)
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from scipy import interpolate
import SINN_functions as sf
import tensorflow as tf

In [29]:
nodes = np.array([[0,0],[1.2,0.1],[0.5,0.9]])
A11 = np.array([[1,0],[0,1]])
A12 = np.array([[0,0],[0,0]])
A22 = np.array([[1,0.5],[0.5,1]])
A = np.concatenate([
    np.concatenate([A11,A12],axis=1),
    np.concatenate([A12,A22],axis=1)
],axis=0)
r = 2

a = np.roll(nodes[:,0],1)*np.roll(nodes[:,1],2) - np.roll(nodes[:,0],2)*np.roll(nodes[:,1],1)
b = np.roll(nodes[:,1],1) - np.roll(nodes[:,1],2)
c = np.roll(nodes[:,0],2) - np.roll(nodes[:,0],1)
Area = np.abs(np.dot(nodes[:,0],b))/2
B = np.concatenate([
    np.concatenate([b[i]*np.eye(r) for i in range(3)],1),
    np.concatenate([c[i]*np.eye(r) for i in range(3)],1)
],0)/(2*Area)
K = np.dot(np.dot(B.T,A),B)*Area

In [33]:
def GetK_el_triang(nodes,A):
    r = int(A.shape[0]/2)
    a = np.roll(nodes[:,0],1)*np.roll(nodes[:,1],2) - np.roll(nodes[:,0],2)*np.roll(nodes[:,1],1)
    b = np.roll(nodes[:,1],1) - np.roll(nodes[:,1],2)
    c = np.roll(nodes[:,0],2) - np.roll(nodes[:,0],1)
    Area = np.abs(np.dot(nodes[:,0],b))/2
    B = np.concatenate([
        np.concatenate([b[i]*np.eye(r) for i in range(3)],1),
        np.concatenate([c[i]*np.eye(r) for i in range(3)],1)
    ],0)/(2*Area)
    return np.dot(np.dot(B.T,A),B)*Area

def SolveFEM(nodes, elements, boundary_nodes, BCfunc, internal_nodes, n_gauss, r, A, A_nl=False, l=None):
    if l is None:
        l = np.zeros((nodes.shape[0], r))
    if not A_nl:
        A_l = A
    # Assemble the global stiffness matrix
    K = np.zeros((nodes.shape[0]*r, nodes.shape[0]*r))
    for el in elements:
        el_idx = [[r*k+j for j in range(r)] for k in el]
        el_idx = np.concatenate(el_idx)
        nodes_el = tf.gather(nodes, indices=el)
        X_idx,Y_idx = np.meshgrid(el_idx,el_idx)
        if A_nl:
            A_l = A(l[el_idx])
        # print(A_l)
        K_el = GetK_el_triang(A_l,nodes_el,n_gauss)
        K[Y_idx,X_idx] += K_el

    # Apply Dirichlet BC
    x_BC = nodes[boundary_nodes,0]
    y_BC = nodes[boundary_nodes,1]
    alpha = np.arctan2(y_BC-0.5,x_BC-0.5)
    l_BC = BCfunc(alpha)
    bc_idx = [[r*i+j for j in range(r)] for i in boundary_nodes]
    bc_idx = np.concatenate(bc_idx)
    internal_idx = [[r*i+j for j in range(r)] for i in internal_nodes]
    internal_idx = np.concatenate(internal_idx)

    f = - (K[:,bc_idx] @ l_BC.flatten().reshape(-1,1))

    K_BC = K[internal_idx,:][:,internal_idx]
    f = f[internal_idx]

    # Solve the system
    l_internal = np.linalg.solve(K_BC, f)
    n_CDOF = int(l_internal.shape[0]/r)
    l_internal = l_internal.reshape(n_CDOF, r)

    l[internal_nodes,:] = l_internal
    l[boundary_nodes,:] = l_BC.reshape(-1,r)
    return l

def PlotFEMsolution(nodes, elements,l):
    # Convert quadrlateral mesh to triangular mesh
    triangles = np.concatenate([elements[:,:3],elements[:,1:]],0)

    # Create a Triangulation object
    triangulation = tri.Triangulation(nodes[:, 0], nodes[:, 1], triangles)

    # Plotting
    plt.figure()
    plt.tricontourf(triangulation, l[:,0],100)
    plt.colorbar()
    # plt.scatter(nodes[:,0],nodes[:,1],s=1,c='k')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [34]:
# Define Dirichlet BC
n_order = 2
p = np.random.randn(2,n_order)/n_order
BCfunc = lambda alpha: np.array([[p[0,j]*np.cos((j+1)*alpha) + p[1,j]*np.sin((j+1)*alpha) for i in range(r)] for j in range(n_order)]).T.sum(axis=-1)

